In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1650793755033_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
from pyspark.sql.functions import col
from pyspark.sql import *
from pyspark.sql.functions import count
from pyspark.sql.functions import when

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# load all the data (includes two springer tables and three AI tables after filtering) for one subfield. e.g. 0101-Pure Mathematics
def loaddata():
    springer_paper_path = "s3://cse6242-hchen703/0101_paper.csv"
    AI_path = "s3://cse6242-hchen703/AIpaperInfo.csv"
    springer_person_path = "s3://cse6242-hchen703/0101_paper_person.csv"
    AI_citation_path = "s3://cse6242-hchen703/citation.csv"
    AI_reference_path = "s3://cse6242-hchen703/reference.csv"
    springerdata = spark.read.csv(springer_paper_path, header=True, inferSchema=True)
    AIdata = spark.read.csv(AI_path, header=True, inferSchema=True)
    springerperson = spark.read.csv(springer_person_path, header=True, inferSchema=True)
    AIcitation = spark.read.csv(AI_citation_path, header=True, inferSchema=True)
    AIreference = spark.read.csv(AI_reference_path, header=True, inferSchema=True)
    return springerdata, AIdata, springerperson, AIcitation, AIreference

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
springerdata, AIdata, springerperson, AIcitation, AIreference= loaddata()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# join the rawdata of AI with filtered springer data
springerdata = springerdata.alias("springer")
AIdata = AIdata.alias("ai")
citation = springerdata.join(AIdata, col("springer.doi") == col("ai.doi")).select(col("springer.doi"),col("paper_name"),col("journal_id"), col("ai.AIid"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# join the rawdata of reference and citation from AI with filtered springer data
AIcitation = AIcitation.alias("AIcitation")
AIreference = AIreference.alias("AIreference")
citation = citation.alias("citation")
citationNeed = citation.join(AIcitation, col("citation.doi") == col("AIcitation.doi")).select(col("citation.doi"), col("cite_AIid"))
referenceNeed = citation.join(AIreference, col("citation.doi") == col("AIreference.doi")).select(col("citation.doi"), col("ref_AIid"))

#transform the AIid of reference paper and citation paper to doi
citationNeed = citationNeed.alias("cite_need")
referenceNeed = referenceNeed.alias("ref_need")
AIdata = AIdata.alias("ai")
ref_transform = referenceNeed.join(AIdata, col("ref_need.ref_AIid") == col("ai.AIid")).select(col("ref_need.doi"), col("ai.doi").alias("ref_doi"))
cite_transform = citationNeed.join(AIdata, col("cite_need.cite_AIid") == col("ai.AIid")).select(col("cite_need.doi"), col("ai.doi").alias("cite_doi"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# focus the citation and reference part to our subfield scale
ref_transform = ref_transform.alias("ref_t")
cite_transform = cite_transform.alias("cite_t")
springerdata = springerdata.alias("springer")
ref = ref_transform.join(springerdata, col("ref_t.ref_doi") == col("springer.doi")).select(col("ref_t.doi"), col("springer.doi").alias("ref_doi"))
cite = cite_transform.join(springerdata, col("cite_t.cite_doi") == col("springer.doi")).select(col("cite_t.doi"), col("springer.doi").alias("cite_doi"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# get paper-to-paper relationship and write the csv file
ref_reorder = ref.select(col("ref_doi").alias("doi"), col("doi").alias("cite_doi"))
relationship = ref_reorder.union(cite).distinct()

bucket = "s3://cse6242-hchen703/project_output/0101paper_relationship"
relationship.coalesce(1).write.option("header","true").csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# add the journal information to the relationship dataframe
springerdata = springerdata.alias("springer")
relationship = relationship.alias("relationship")
add_journal1 = relationship.join(springerdata, col("relationship.doi") == col("springer.doi")).select(col("relationship.doi").alias("doi"), col("springer.journal_id").alias("journal_id"), col("relationship.cite_doi"))
add_journal1 = add_journal1.alias("add_journal1")
add_journal2 = add_journal1.join(springerdata, col("add_journal1.cite_doi") == col("springer.doi")).select(col("add_journal1.journal_id").alias("journal_id"), col("springer.journal_id").alias("cite_journal_id"))
journal_relationship = add_journal2.groupBy(col("journal_id"), col("cite_journal_id")).agg(count("*").alias("cite_num"))
journal = journal_relationship.filter(journal_relationship.journal_id.like('sg%')).filter(journal_relationship.cite_journal_id.like('sg%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# filter journal-to-journal dataframe to those with clear snip and sjr info and get the final journal-to-journal relationship csv file
raw_journal = spark.read.csv("s3://cse6242-hchen703/6242projetc_journal/journal.csv", header=True, inferSchema=True)
raw_journal = raw_journal.alias("raw")
journal = journal.alias("journal")
journal_filter1 = journal.join(raw_journal, col("journal.journal_id") == col("raw.journal_id")).select(col("raw.journal_id"), col("journal.cite_journal_id"), col("journal.cite_num"))
journal_filter1 = journal_filter1.alias("filter")
journal_filter2 = journal_filter1.join(raw_journal, col("filter.cite_journal_id") == col("raw.journal_id")).select(col("filter.journal_id"), col("filter.cite_journal_id"), col("filter.cite_num"))

bucket2 = "s3://cse6242-hchen703/project_output/0101journal_relationship"
journal_filter2.coalesce(1).write.option("header","true").csv(bucket2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…